# Download da base de dados do Supabase

In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
from supabase import create_client



load_dotenv()

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")
pd.set_option('display.max_columns', None)

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
query = supabase.table("wine_data").select("*").execute()
db_original = pd.DataFrame(query.data)

In [ ]:
db = db_original.copy()

# Informações básica do DataFrame

In [190]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 32 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   id                                   623 non-null    int64 
 1   product_type                         584 non-null    object
 2   product_name                         623 non-null    object
 3   wine_variety                         584 non-null    object
 4   wine_region                          623 non-null    object
 5   item_quantity                        623 non-null    object
 6   wine_grapes                          281 non-null    object
 7   color_description                    421 non-null    object
 8   scent_description                    420 non-null    object
 9   taste_description                    422 non-null    object
 10  fruit_tasting                        422 non-null    object
 11  sugar_tasting                        422 non-

In [191]:
db.describe()

,id
count,623.000000
mean,312.410915
std,180.671981
min,1.000000
25%,156.500000
50%,312.000000
75%,467.500000
max,631.000000


# Feature Engineering

## Feature: Item quantity -> int 

### Resultado: Foi criada uma feature com o numero de unidade com o tipo int o nome da coluna é: item_quantity_integer

In [192]:
teste = db['item_quantity'].sort_values(ascending=False, key=lambda x: x.str.len())
teste

14     10 unidades
25     10 unidades
573    10 unidades
268    10 unidades
277    10 unidades
          ...     
296              1
295              1
292              1
290              1
287              1
Name: item_quantity, Length: 623, dtype: object

In [193]:
db['item_quantity_integer'] = db['item_quantity'].sort_values(ascending=False, key=lambda x: x.str.len())
db['item_quantity_integer'] = db['item_quantity'].str.split(" ").apply(lambda x: x[0]).astype(int)
db['item_quantity_integer']

0      1
1      1
2      6
3      4
4      6
      ..
618    3
619    1
620    3
621    1
622    1
Name: item_quantity_integer, Length: 623, dtype: int64

In [194]:
db['item_quantity_integer'].max()

np.int64(12)

In [195]:
db.drop("item_quantity", axis='columns', inplace=True)

## Feature: wine_variety -> Retirando os product type nulos. Não são vinhos

### Resultado: As entradas foram retiradas do dataframe que irá pro modelo por possuir uma variedade de itens que não são vinhos

In [196]:
db["wine_variety"].unique()

array(['Vinho branco', 'Vinho tinto', 'Vários tipos', 'Vinho rosé',
       'Espumante Branco', None, 'Espumante Rosé'], dtype=object)

In [198]:
db[db['wine_variety'].isna()].head()

,id,product_type,product_name,wine_variety,wine_region,wine_grapes,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer
26,27,None,Baco | O Jogo dos Vinhos,None,Brasil,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T17:59:51.461769,None,None,None,None,1
30,55,None,Larios 12 Premium Gin Mediterránea - 700 mL,None,Espanha,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T18:07:30.180141,None,None,None,https://res.cloudinary.com/evino/image/upload/...,1
33,22,None,Old Fashioned Bottled Cocktail - 375 mL,None,Brasil,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T17:58:37.692023,None,None,None,https://res.cloudinary.com/evino/image/upload/...,1
38,61,None,Bolsa Térmica Exclusiva Evino para 1 garrafa,None,Brasil,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T18:09:58.576572,None,None,None,https://res.cloudinary.com/evino/image/upload/...,1
45,72,None,Pôster Regiões Vinícolas da Espanha,None,Brasil,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T18:32:59.866272,None,None,None,https://res.cloudinary.com/evino/image/upload/...,1


In [199]:
db = db[db["wine_variety"].isna()==False]

## Feature: product_type 

### Resultado: Os kits são conjuntos de produtos cujas páginas da Evino possuem muito menos informação, o que não é bom para o modelo. Desta forma, foi necessário retirar e criar um dataframe a parte para caso queiramos utilizar mais a frente.

In [200]:
db[db["product_type"]=="Kit"]

,id,product_type,product_name,wine_variety,wine_region,wine_grapes,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer
2,11,Kit,Kit Primitivos Best Sellers | 6 garrafas por R...,Vinho tinto,Itália,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T02:37:57.406997,None,None,None,None,6
3,1,Kit,Kit 3 Malbecs Best Sellers + Bolsa Térmica,Vários tipos,Vários países,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T03:18:12.856299,None,None,None,None,4
4,4,Kit,Kit Linha Vini de Angeli | 2 Primitivo + 2 Pin...,Vários tipos,Itália,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T03:18:27.095948,None,None,None,None,6
8,16,Kit,Kit Première Bulle | 2 Branco + 2 Rosé por R$1...,Vários tipos,França,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T17:56:44.828058,None,None,None,None,4
11,18,Kit,Kit Vinhos Premiados Best Sellers | 5 garrafas...,Vinho tinto,Vários países,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T17:57:23.282315,None,None,None,None,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,447,Kit,Kit Linha Atardecer de Los Andes + Kit 6 Marca...,Vários tipos,Vários países,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T00:55:01.854817,None,None,None,None,5
573,550,Kit,"Kit 10 Vinhos Tintos por R$22,90 cada garrafa*",Vinho tinto,Vários países,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T01:59:49.594106,None,None,None,None,10
594,25,Kit,"Kit 3 Espumantes Best Sellers por R$109,90 cad...",Vários tipos,Vários países,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T02:38:13.208747,None,None,None,None,3
600,102,Kit,Kit Primitivos di Manduria | 2 Casale Brondell...,Vinho tinto,Itália,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T02:41:25.57598,None,None,None,None,4


In [201]:
db_kits = db[db['product_type']=="Kit"]
db = db[db['product_type']!="Kit"]

In [202]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 431 entries, 0 to 622
Data columns (total 32 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   id                                   431 non-null    int64 
 1   product_type                         431 non-null    object
 2   product_name                         431 non-null    object
 3   wine_variety                         431 non-null    object
 4   wine_region                          431 non-null    object
 5   wine_grapes                          281 non-null    object
 6   color_description                    421 non-null    object
 7   scent_description                    420 non-null    object
 8   taste_description                    422 non-null    object
 9   fruit_tasting                        422 non-null    object
 10  sugar_tasting                        422 non-null    object
 11  acidity_tasting                      422 non-null 

## Feature wine_grapes -> Avaliação

### Resultado: Talvez seja improtante retirar essas entradas do dataframe principal, assim como fiz com os kits e separá-las num outro df que juntarei com o df dos kits. 

In [203]:
db["wine_grapes"].unique()

array(['Sauvignon Blanc', 'Malbec', None, 'Uvas variadas',
       'Cabernet Sauvignon', 'Jaén, Touriga Nacional, Aragonez', 'Blend',
       'Tempranillo, Merlot, Syrah', 'Carménère', 'Montepulciano',
       'Tempranillo', 'Merlot', 'Torrontés', 'Pinot Grigio',
       'Arinto, Trajadura', 'Chardonnay', 'Garnacha, Syrah, Tempranillo',
       'Pinot Noir', 'Bobal', 'Primitivo', 'Blend, Tempranillo',
       'Garnacha, Tempranillo', 'Nebbiolo', 'Garnacha',
       'Grenache, Syrah, Tempranillo',
       'Blend, Chardonnay, Chenin Blanc, Torrontés', 'Syrah', 'Airén',
       'Barbera, Dolcetto, Freisa, Bonarda, Albarossa, Merlot, Syrah, Cabernet Sauvignon',
       'Blend, Negroamaro, Primitivo, Uvas variadas', 'Moscatel',
       'Touriga Franca, Tinta Roriz', 'Blend, Seara Nova, Vital',
       'Bobal, Tempranillo',
       'Monastrell, Cabernet Sauvignon, Tempranillo', 'Sangiovese',
       'Merlot, Cabernet Sauvignon', 'Grenache, Mourvèdre, Syrah',
       'Merlot, Cabernet Sauvignon, Primitivo, 

In [204]:
db[db["wine_grapes"].isna()]

,id,product_type,product_name,wine_variety,wine_region,wine_grapes,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer
5,5,Vinho tinto italiano,Kit 6 C.O.S 1960 Primitivo di Manduria DOP por...,Vinho tinto,Itália,None,Vermelho-rubi intenso com reflexos violáceos,"Intenso, com notas de frutas vermelhas e preta...","Rico, encorpado, sedoso e frutado, com taninos...",4,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,18ºC,Itália,Puglia,14%,Primitivo,Cantolio,2022,2025,None,2025-03-24T03:18:40.820898,None,Vinícius Santiago,Sommelier da evino,None,6
7,10,Vinho tinto chileno,Kit 6 Reyna Cabernet Sauvignon Central Valley ...,Vinho tinto,Chile,None,Vermelho-rubi intenso,"Notas intensas de figos maduros e azeitonas, c...","Rico, macio e elegante",4,1,2,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,18ºC,Chile,Valle Central,12.5%,Cabernet Sauvignon,Bodega Tagua Tagua,2023,2025,None,2025-03-24T03:19:08.864529,None,Vinícius Santiago,Sommelier da evino,None,6
20,40,Vinho tinto sul-africano,Kit 3 The African Horizon Pinotage Western Cap...,Vinho tinto,África do Sul,None,Vermelho-rubi,"Notas de frutas vermelhas, como cereja e grose...","Frutado e suculento, ressaltando taninos macio...",3,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Tampa de rosca,15ºC,África do Sul,Western Cape,13%,Pinotage,Origin Wine,2023,2025,None,2025-03-23T18:03:31.029735,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,3
22,44,Espumante Branco brasileiro,"Kit 6 Alísios Brut por R$74,90 cada garrafa",Espumante Branco,Brasil,None,Amarelo pálido com reflexos esverdeados,"Aromas intensos de frutas tropicais e flores, ...","Equilibrado, refrescante e agradável, com perl...",4,1,4,1,"Carnes brancas, Frutos do mar, Queijos, Risoto...",Espumante Branco,750ml,Rolha de cortiça,8ºC,Brasil,Vale do São Francisco,12%,"Verdejo, Sauvignon Blanc, Chenin Blanc",Miolo Wine Group,None,2025,None,2025-03-23T18:04:43.75337,None,Ari Gorenstein,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,6
28,49,Vinho tinto espanhol,Kit 6 Valtier Reserva Utiel-Requena DOP por R$...,Vinho tinto,Espanha,None,Vermelho-rubi com reflexos violáceos,"Ameixa madura, pimenta, tostado e especiarias","Encorpado, equilibrado, taninos macios e redondos",4,1,3,3,"Carnes de caça, Carnes vermelhas, Pizzas e mas...",Vinho tinto,750ml,Rolha de cortiça,16ºC,Espanha,Utiel-Requena,13%,"Tempranillo, Bobal",Marqués del Atrio,2018,2025,Estágio em barricas de carvalho americano,2025-03-23T18:06:16.595191,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,285,Vinho tinto italiano,Kit 4 Infinitum Montepulciano d'Abruzzo DOC po...,Vinho tinto,Itália,None,Vermelho-rubi,Notas de cerejas maduras e amoras,"Equilibrado e encorpado, com taninos macios",3,1,3,2,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha,16ºC,Itália,Abruzzo,12.5%,Montepulciano,Deangeli Musti Nobilis,None,2025,None,2025-03-24T03:20:25.177931,None,Vinícius Santiago,Sommelier da evino,None,4
616,306,Vinho tinto italiano,Kit 3 Conte Parelli Appassimento Puglia IGT po...,Vinho tinto,Itália,None,Vermelho-rubi intenso com reflexos violáceos,"Notas intensas de frutas vermelhas 

In [205]:
db["product_name_with_word_kit"] = db['product_name'].str.contains("Kit", case=False)

In [206]:
db[db["product_name_with_word_kit"]==True].info()

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 5 to 620
Data columns (total 33 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   id                                   150 non-null    int64 
 1   product_type                         150 non-null    object
 2   product_name                         150 non-null    object
 3   wine_variety                         150 non-null    object
 4   wine_region                          150 non-null    object
 5   wine_grapes                          0 non-null      object
 6   color_description                    148 non-null    object
 7   scent_description                    147 non-null    object
 8   taste_description                    148 non-null    object
 9   fruit_tasting                        148 non-null    object
 10  sugar_tasting                        148 non-null    object
 11  acidity_tasting                      148 non-null 

In [207]:
pd.set_option('display.max_columns', None)
db[db["product_name_with_word_kit"]==True]

,id,product_type,product_name,wine_variety,wine_region,wine_grapes,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer,product_name_with_word_kit
5,5,Vinho tinto italiano,Kit 6 C.O.S 1960 Primitivo di Manduria DOP por...,Vinho tinto,Itália,None,Vermelho-rubi intenso com reflexos violáceos,"Intenso, com notas de frutas vermelhas e preta...","Rico, encorpado, sedoso e frutado, com taninos...",4,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,18ºC,Itália,Puglia,14%,Primitivo,Cantolio,2022,2025,None,2025-03-24T03:18:40.820898,None,Vinícius Santiago,Sommelier da evino,None,6,True
7,10,Vinho tinto chileno,Kit 6 Reyna Cabernet Sauvignon Central Valley ...,Vinho tinto,Chile,None,Vermelho-rubi intenso,"Notas intensas de figos maduros e azeitonas, c...","Rico, macio e elegante",4,1,2,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,18ºC,Chile,Valle Central,12.5%,Cabernet Sauvignon,Bodega Tagua Tagua,2023,2025,None,2025-03-24T03:19:08.864529,None,Vinícius Santiago,Sommelier da evino,None,6,True
20,40,Vinho tinto sul-africano,Kit 3 The African Horizon Pinotage Western Cap...,Vinho tinto,África do Sul,None,Vermelho-rubi,"Notas de frutas vermelhas, como cereja e grose...","Frutado e suculento, ressaltando taninos macio...",3,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Tampa de rosca,15ºC,África do Sul,Western Cape,13%,Pinotage,Origin Wine,2023,2025,None,2025-03-23T18:03:31.029735,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,3,True
22,44,Espumante Branco brasileiro,"Kit 6 Alísios Brut por R$74,90 cada garrafa",Espumante Branco,Brasil,None,Amarelo pálido com reflexos esverdeados,"Aromas intensos de frutas tropicais e flores, ...","Equilibrado, refrescante e agradável, com perl...",4,1,4,1,"Carnes brancas, Frutos do mar, Queijos, Risoto...",Espumante Branco,750ml,Rolha de cortiça,8ºC,Brasil,Vale do São Francisco,12%,"Verdejo, Sauvignon Blanc, Chenin Blanc",Miolo Wine Group,None,2025,None,2025-03-23T18:04:43.75337,None,Ari Gorenstein,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,6,True
28,49,Vinho tinto espanhol,Kit 6 Valtier Reserva Utiel-Requena DOP por R$...,Vinho tinto,Espanha,None,Vermelho-rubi com reflexos violáceos,"Ameixa madura, pimenta, tostado e especiarias","Encorpado, equilibrado, taninos macios e redondos",4,1,3,3,"Carnes de caça, Carnes vermelhas, Pizzas e mas...",Vinho tinto,750ml,Rolha de cortiça,16ºC,Espanha,Utiel-Requena,13%,"Tempranillo, Bobal",Marqués del Atrio,2018,2025,Estágio em barricas de carvalho americano,2025-03-23T18:06:16.595191,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,6,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,285,Vinho tinto italiano,Kit 4 Infinitum Montepulciano d'Abruzzo DOC po...,Vinho tinto,Itália,None,Vermelho-rubi,Notas de cerejas maduras e amoras,"Equilibrado e encorpado, com taninos macios",3,1,3,2,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha,16ºC,Itália,Abruzzo,12.5%,Montepulciano,Deangeli Musti Nobilis,None,2025,None,2025-03-24T03:20:25.177931,None,Vinícius Santiago,Sommelier da evino,None,4,True
616,306,Vinho tinto italiano,Kit 3 Conte Parelli Appassimento Puglia IGT po...,Vinho tinto,Itália,None,Vermelho-rubi intens

In [208]:
db[db["product_name"].str.contains("African Horizon Pinotage Western")]

,id,product_type,product_name,wine_variety,wine_region,wine_grapes,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer,product_name_with_word_kit
20,40,Vinho tinto sul-africano,Kit 3 The African Horizon Pinotage Western Cap...,Vinho tinto,África do Sul,None,Vermelho-rubi,"Notas de frutas vermelhas, como cereja e grose...","Frutado e suculento, ressaltando taninos macio...",3,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Tampa de rosca,15ºC,África do Sul,Western Cape,13%,Pinotage,Origin Wine,2023,2025,None,2025-03-23T18:03:31.029735,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,3,True
570,543,Vinho tinto sul-africano,The African Horizon Pinotage Western Cape W.O....,Vinho tinto,África do Sul•,Pinotage,Vermelho-rubi,"Notas de frutas vermelhas, como cereja e grose...","Frutado e suculento, ressaltando taninos macio...",3,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Tampa de rosca,15ºC,África do Sul,Western Cape,13%,Pinotage,Origin Wine,2023,2025,None,2025-03-24T01:58:16.95541,"Uva tinta mais famosa da África do Sul, a Pino...",Vinícius Santiago,Sommelier da evino,None,1,False


Observação: 
Como as entradas são identicas mas com menos volume de informações, por exemplo: em kits de produtos tem as avaliações de força de acidez, nível de fruta e etc mas não tem a avaliação do especialista. Talvez seja improtante retirar essas entradas do dataframe principal, assim como fiz com os kits e separá-las num outro df que juntarei com o df dos kits.

In [209]:
db_kits_from_product_name = db[db['product_name_with_word_kit']==True]
db = db[db['product_name_with_word_kit']==False]


db_all_kits = pd.concat([db_kits, db_kits_from_product_name], axis=0,ignore_index=True)

In [210]:
db_all_kits

,id,product_type,product_name,wine_variety,wine_region,wine_grapes,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer,product_name_with_word_kit
0,11,Kit,Kit Primitivos Best Sellers | 6 garrafas por R...,Vinho tinto,Itália,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T02:37:57.406997,None,None,None,None,6,NaN
1,1,Kit,Kit 3 Malbecs Best Sellers + Bolsa Térmica,Vários tipos,Vários países,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T03:18:12.856299,None,None,None,None,4,NaN
2,4,Kit,Kit Linha Vini de Angeli | 2 Primitivo + 2 Pin...,Vários tipos,Itália,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T03:18:27.095948,None,None,None,None,6,NaN
3,16,Kit,Kit Première Bulle | 2 Branco + 2 Rosé por R$1...,Vários tipos,França,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T17:56:44.828058,None,None,None,None,4,NaN
4,18,Kit,Kit Vinhos Premiados Best Sellers | 5 garrafas...,Vinho tinto,Vários países,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T17:57:23.282315,None,None,None,None,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,285,Vinho tinto italiano,Kit 4 Infinitum Montepulciano d'Abruzzo DOC po...,Vinho tinto,Itália,None,Vermelho-rubi,Notas de cerejas maduras e amoras,"Equilibrado e encorpado, com taninos macios",3,1,3,2,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha,16ºC,Itália,Abruzzo,12.5%,Montepulciano,Deangeli Musti Nobilis,None,2025,None,2025-03-24T03:20:25.177931,None,Vinícius Santiago,Sommelier da evino,None,4,True
299,306,Vinho tinto italiano,Kit 3 Conte Parelli Appassimento Puglia IGT po...,Vinho tinto,Itália,None,Vermelho-rubi intenso com reflexos violáceos,"Notas intensas de frutas vermelhas maduras, e ...","Elegante e harmonioso, com taninos aveludados ...",4,2,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,17ºC,Itália,Puglia,15%,"Blend, Negroamaro, Uvas variadas",Angelo Rocca e Figli Srl,2021,2025,Estágio em barricas de carvalho francês,2025-03-24T03:21:06.626014,None,Vinícius Santiago,Sommelier da evino,None,3,True
300,354,Espumante Branco francês,Kit 4 Première Bulle Blanquette de Limoux AOC ...,Espumante Branco,França,None,Dourado pálido com perlages finos,Aromas delicados de frutas cítricas e flores b...,"Vivaz e redondo, com textura agradável, notas ...",3,1,4,1,"Carnes brancas, Frutos do mar, Risoto e massas...",Espumante Branco,750ml,Rolha de cortiça,6ºC,França,Languedoc-Roussillon,12.5%,"Mauzac, Chardonnay, Chenin Blanc",Sieur d'Arques,None,2025,Envelhecimento sobre as borras por 18 meses,2025-03-24T03:22:08.758223,None,Vinícius Santiago,Sommelier da evino,None,4,True
301,383,Vinho branco sul-africano,Kit 3 The African Horizon Chenin Blanc Western...,Vinho branco,África do Sul,None,Amarelo-palha,"Aromas frescos de frutas cítricas, de caroço e...","Leve e refrescante, com bom equilíbrio entre f...",3,1,4,1,"Carnes brancas, Frutos do mar, Queijos, Salada...",Vinho branco,750ml,Tampa de rosca,6ºC,África do Sul,Western Cape,12%,Chenin Blanc,Origin Wine,2023,2025,None,2025-03-24T03:22:51.982845

## Features: wine_grapes e technical_sheet_grapes

### Resultado: Podemos desconsiderar uma das duas colunas dado que os valores são todos iguais.

In [211]:
db[['wine_grapes', "technical_sheet_grapes"]]

,wine_grapes,technical_sheet_grapes
0,Sauvignon Blanc,Sauvignon Blanc
1,Malbec,Malbec
6,Uvas variadas,Uvas variadas
9,Cabernet Sauvignon,Cabernet Sauvignon
10,Uvas variadas,Uvas variadas
...,...,...
613,Malbec,Malbec
614,Grenache,Grenache
619,Nebbiolo,Nebbiolo
621,Blend,Blend


In [212]:
db['wine_grapes_equal_tech_sheet'] = db['wine_grapes']==db['technical_sheet_grapes']
db['wine_grapes_equal_tech_sheet'].value_counts()

wine_grapes_equal_tech_sheet
True    281
Name: count, dtype: int64

In [213]:
db.drop("wine_grapes", axis='columns', inplace=True)

Podemos desconsiderar uma das duas colunas dado que os valores são todos iguais.

## Features: wine_variety e technical_sheet_wine_type

### Resultado: A variável product_type é uma informação mais detalhada das outras duas variáveis

In [214]:
db[["product_type", "wine_variety", "technical_sheet_wine_type"]]

,product_type,wine_variety,technical_sheet_wine_type
0,Vinho branco sul-africano,Vinho branco,Vinho branco
1,Vinho tinto argentino,Vinho tinto,Vinho tinto
6,Vinho tinto chileno,Vinho tinto,Vinho tinto
9,Vinho tinto chileno,Vinho tinto,Vinho tinto
10,Vinho tinto argentino,Vinho tinto,Vinho tinto
...,...,...,...
613,Vinho tinto argentino,Vinho tinto,Vinho tinto
614,Espumante Rosé brasileiro,Espumante Rosé,Espumante Rosé
619,Vinho tinto italiano,Vinho tinto,Vinho tinto
621,Vinho tinto chileno,Vinho tinto,Vinho tinto


In [215]:
db["product_type"].unique()

array(['Vinho branco sul-africano', 'Vinho tinto argentino',
       'Vinho tinto chileno', 'Vinho tinto português',
       'Vinho tinto europeu', 'Vinho rosé europeu', 'Vinho rosé chileno',
       'Vinho tinto italiano', 'Vinho tinto espanhol',
       'Vinho rosé argentino', 'Vinho branco chileno',
       'Espumante Branco espanhol', 'Vinho rosé espanhol',
       'Vinho branco argentino', 'Vinho branco italiano',
       'Vinho branco português', 'Espumante Branco brasileiro',
       'Vinho branco espanhol', 'Vinho tinto francês',
       'Vinho rosé sul-africano', 'Vinho rosé português',
       'Vinho rosé italiano', 'Espumante Branco francês',
       'Espumante Rosé francês', 'Vinho branco francês',
       'Espumante Rosé espanhol', 'Espumante Branco argentino',
       'Espumante Rosé brasileiro', 'Vinho rosé francês',
       'Vinho tinto sul-africano', 'Vinho branco brasileiro'],
      dtype=object)

A variável product_type é uma informação mais detalhada das outras duas variáveis

In [216]:
db['wine_variety_equal_tech_sheet'] = db['wine_variety']==db['technical_sheet_wine_type']
db['wine_grapes_equal_tech_sheet'].value_counts()

wine_grapes_equal_tech_sheet
True    281
Name: count, dtype: int64

In [217]:
db.drop("wine_variety", axis='columns', inplace=True)

Também podemos desconsiderar uma das duas variáveis dado que elas são iguais.

## Features: wine_region e technical_sheet_country

### Resultado: Podemos desconsiderar uma das duas variáveis 'wine_region' ou'technical_sheet_country' dado que são iguais.

In [218]:
db[['wine_region', 'technical_sheet_region', 'technical_sheet_country']].head(5)

,wine_region,technical_sheet_region,technical_sheet_country
0,África do Sul•,Western Cape,África do Sul
1,Argentina•,Mendoza,Argentina
6,Chile•,Valle del Maule,Chile
9,Chile•,Valle Central,Chile
10,Argentina•,Mendoza,Argentina


In [219]:
db['wine_region'] = db['wine_region'].str.replace("•", "")

In [220]:
db[['wine_region','technical_sheet_country']].value_counts()

wine_region    technical_sheet_country
Argentina      Argentina                  70
Chile          Chile                      65
Espanha        Espanha                    47
Itália         Itália                     34
Portugal       Portugal                   20
França         França                     15
Europeu        Europeu                    14
Brasil         Brasil                     11
África do Sul  África do Sul               5
Name: count, dtype: int64

In [221]:
db['wine_region_equal_tech_sheet'] = db['wine_region']==db['technical_sheet_country']
db['wine_region_equal_tech_sheet'].value_counts()

wine_region_equal_tech_sheet
True    281
Name: count, dtype: int64

In [222]:
db[['wine_region','technical_sheet_country']].head(5)

,wine_region,technical_sheet_country
0,África do Sul,África do Sul
1,Argentina,Argentina
6,Chile,Chile
9,Chile,Chile
10,Argentina,Argentina


In [223]:
db.drop("wine_region", axis='columns', inplace=True)

Podemos desconsiderar uma das duas variáveis 'wine_region' ou'technical_sheet_country' dado que são iguais.

## Features: fruit_tasting, tannin_tasting, sugar_tasting e acidity_tasting


### Resultado: Como todos os itens que não possuem avaliação são os mesmos, a melhor alternativa é tirá-los para não serem um problema no modelo de machine learning.

In [224]:
db.loc[db['fruit_tasting'].isna()==True]['id'].tolist()

[238, 261, 460, 499, 559, 557, 42]

In [225]:
db.loc[db['tannin_tasting'].isna()==True]['id'].tolist()

[238, 261, 460, 499, 559, 557, 42]

In [226]:
db.loc[db['sugar_tasting'].isna()==True]['id'].tolist()

[238, 261, 460, 499, 559, 557, 42]

In [227]:
db.loc[db['acidity_tasting'].isna()==True]['id'].tolist()

[238, 261, 460, 499, 559, 557, 42]

Como todos os itens que não possuem avaliação são os mesmos, a melhor alternativa é tirá-los para não serem um problema no modelo de machine learning.

In [228]:
db = db.loc[db['acidity_tasting'].isna()==False]

In [229]:
db['id'].count()

np.int64(274)

## Feature: product_name

### Resultado: Após as exclusões anteriores, agora esta feature não possui mais nomes "sujos", portanto não será necessário realizar nenhum tipo de limpeza aqui.

In [244]:
db['product_name'].unique()

array(['The African Horizon Sauvignon Blanc Western Cape W.O. 2023',
       'Villa del Nevado Malbec 2024', 'Candle Single Vineyard Tinto',
       'Lagunas Cabernet Sauvignon Valle Central D.O. 2023',
       'Rocas Viejas Red Blend 2024',
       'Foral D. Henrique Reserva Dão DOC 2021',
       'Viña de Los Andes Red Blend 2023', 'Vigneto di Carla 2023',
       'Vivid Rosé', 'Nouaison Rouge',
       'Concha y Toro Exportacion Selecto Cabernet Sauvignon 2024',
       'Pioneers Rosé 2024', "Infinitum Montepulciano d'Abruzzo DOC 2021",
       'Torre Oria Crianza Utiel-Requena D.O. 2022',
       'Viña de Los Andes Rosé 2024',
       'Selection 92 Reserva Sauvignon Blanc Valle Central 2024',
       'Yannet Selection Vin Rouge 2023',
       'El Emperador Cabernet Sauvignon Valle Central 2022',
       'Las Colinas De Los Andes Red Blend 2024', 'Tanggier Brut',
       'Viña de Los Andes Malbec 2023', 'Emparrado Blend de Tintas',
       'San Nazareno Winemaker Selection Rosé 2024',
       'Monte

Observação: Essa feature contém os nomes corretos, uma vez que tiramos os kits e demais sujeiras do dataset. Não é necessário fazer nada aqui.

## Feature: temperature

In [ ]:
db["technical_sheet_service_temperature"] = db["technical_sheet_service_temperature"].str.replace("ºC", "")
db.rename(columns={'technical_sheet_service_temperature': 'technical_sheet_service_temperature_in_celsius'}, inplace=True)
db['technical_sheet_service_temperature_in_celsius']

0       6
1      18
6      16
9      18
10     16
       ..
613    18
614     9
619    17
621    16
622    18
Name: technical_sheet_service_temperature_in_celsius, Length: 274, dtype: object

## Feature: technical_sheet_alcohol_content

In [ ]:
db["technical_sheet_alcohol_content"] = db["technical_sheet_alcohol_content"].str.replace("%","")
db["technical_sheet_alcohol_content"]

0        12
1        13
6      12.5
9      12.5
10       12
       ... 
613    12.5
614      12
619      14
621    12.5
622    13.5
Name: technical_sheet_alcohol_content, Length: 274, dtype: object

## Features: fruit_tasting,sugar_tasting, acidity_tasting, tannin_tasting

In [ ]:
db[['fruit_tasting','sugar_tasting', 
    'acidity_tasting', 'tannin_tasting']] = db[['fruit_tasting','sugar_tasting', 
                                                'acidity_tasting', 'tannin_tasting']].astype(int)

## Ajustando o DataFrame pós análises, preparando para os próximos passos.

In [ ]:
db.drop(["created_at", "photo_url", "product_name_with_word_kit", 
         "wine_grapes_equal_tech_sheet", "wine_variety_equal_tech_sheet", 
         "wine_region_equal_tech_sheet", "item_quantity_integer"], axis='columns', inplace=True)

Observação: Como retirei todos os kits, agora a informação item_quanty_integer não sei se vale de tanto e é passível de exclusão também.

In [235]:
db.head(10)

,id,product_type,product_name,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,specialist_review_content,specialist_review_owner,specialist_review_occupation
0,355,Vinho branco sul-africano,The African Horizon Sauvignon Blanc Western Ca...,Amarelo-palha brilhante,"Aromas frutados e notas de maracujá, pêssego e...","Saboroso, equilibrado e de final longo, com to...",4,1,4,1,"Carnes brancas, Frutos do mar, Queijos, Risoto...",Vinho branco,750ml,Tampa de rosca,6ºC,África do Sul,Western Cape,12%,Sauvignon Blanc,Origin Wine,2023,2025,None,"Elaborada em Western Cape, na África do Sul, e...",Vinícius Santiago,Sommelier da evino
1,419,Vinho tinto argentino,Villa del Nevado Malbec 2024,Vermelho-rubi brilhante e profundo,"Intenso, com notas de frutas maduras e leves t...","Equilibrado, harmonioso, com taninos redondos,...",4,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,18ºC,Argentina,Mendoza,13%,Malbec,Fecovita,2024,2025,None,None,Vinícius Santiago,Sommelier da evino
6,9,Vinho tinto chileno,Candle Single Vineyard Tinto,Vermelho intenso,Frutas vermelhas maduras,"Macio, frutado, fácil de beber e fresco",4,1,3,2,"Pizzas e massas de molho vermelho, Queijos",Vinho tinto,750ml,Rolha,16ºC,Chile,Valle del Maule,12.5%,Uvas variadas,Aura Wines,None,2025,None,None,Ari Gorenstein,Sommelier da evino
9,7,Vinho tinto chileno,Lagunas Cabernet Sauvignon Valle Central D.O. ...,Vermelho-rubi intenso e profundo,"Expressivo, com notas de frutas pretas, grosel...","Saboroso, harmônico, equilibrado, com taninos ...",4,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,18ºC,Chile,Valle Central,12.5%,Cabernet Sauvignon,Cremaschi Furlotti,2023,2025,None,None,Vinícius Santiago,Sommelier da evino
10,13,Vinho tinto argentino,Rocas Viejas Red Blend 2024,Vermelho-rubi,Frutas vermelhas frescas como cereja e morango,"Bastante sabor de frutas, boa acidez e maciez",3,1,3,3,"Carnes brancas, Carnes vermelhas, Pizzas e mas...",Vinho tinto,750ml,Rolha de cortiça,16ºC,Argentina,Mendoza,12%,Uvas variadas,Fecovita,2024,2025,None,None,Vinícius Santiago,Sommelier da evino
12,3,Vinho tinto português,Foral D. Henrique Reserva Dão DOC 2021,Vermelho-rubi,Aroma intenso de compota de frutos silvestres ...,"Paladar equilibrado, taninos sedosos e final l...",3,1,3,3,"Carnes de caça, Carnes vermelhas, Queijos",Vinho tinto,750ml,Rolha,18ºC,Portugal,Dão,12.5%,"Jaén, Touriga Nacional, Aragonez",Adega Cooperativa Mangualde,2021,2025,6 meses em tanques de aço inoxidável e barrica...,None,Ari Gorenstein,Sommelier da evino
13,19,Vinho tinto argentino,Viña de Los Andes Red Blend 2023,Vermelho-rubi,"Aromas suaves, frutados e frescos","Rico, marcante e com taninos suaves",3,2,2,2,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,16ºC,Argentina,Mendoza,12.5%,Uvas variadas,Fecovita,2023,2025,None,None,Vinícius Santiago,Sommelier da evino
15,6,Vinho tinto europeu,Vigneto di Carla 2023,Vermelho-rubi,"Aromas de frutas vermelhas, como morango e cereja","Frutado, equilibrado e fácil de beber, com tan...",3,1,2,2,"Carnes brancas, Pizzas e massas de molho verme...",Vinho tinto,750ml,Rolha,16ºC,Europeu,Multiregional,11%,Blend,Domaine du Père Guillot,2023,2025,None,None,Vinícius Santiago,Sommelier da evino
16,421,Vinho rosé europeu,Vivid Rosé,Rosa claro e translúcido,Frutas vermelhas frescas,"Doçura agradável, frutado, equilibrado e persi...",3,5,2,1,"Saladas e aperitivos, Sobremesas",Vinho rosé,750ml,Tampa de rosca,16ºC,Europeu,None,10.5%,"Tempranillo, Merlot, Syrah",Castel,None,20

In [265]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 274 entries, 0 to 622
Data columns (total 27 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   id                                   274 non-null    int64 
 1   product_type                         274 non-null    object
 2   product_name                         274 non-null    object
 3   color_description                    273 non-null    object
 4   scent_description                    273 non-null    object
 5   taste_description                    274 non-null    object
 6   fruit_tasting                        274 non-null    int64 
 7   sugar_tasting                        274 non-null    int64 
 8   acidity_tasting                      274 non-null    int64 
 9   tannin_tasting                       274 non-null    int64 
 10  harmonizes_with                      263 non-null    object
 11  technical_sheet_wine_type            274 non-null 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

class WineRecommender:
    def __init__(self, dataframe):
        self.df = dataframe
        self.prepare_features()
    
    def prepare_features(self):
        """Definir aqui quais colunas serão usadas para definir a similaridade, 
           podemos, no limite, botar todas as caracteristicas que quisermos, basta
           apenas separar entre o que é texto e o que é numérico.
        """
        self.text_columns = [
            'product_name', 
            'color_description', 
            'scent_description', 
            'taste_description',
            'harmonizes_with',
            'technical_sheet_wine_type',
            'technical_sheet_grapes',
            'technical_sheet_region',
            'technical_sheet_country'
        ]
        
        self.numeric_columns = [
            'fruit_tasting', 
            'sugar_tasting', 
            'acidity_tasting', 
            'tannin_tasting',
        ]
        
        # Lidamos com nulos e realizamos um join em todas as características de texto em uma coluna.
        self.df['combined_text_features'] = self.df[self.text_columns].fillna('').apply(lambda x: ' '.join(x.astype(str)), axis=1)
        
        # Ralizamos uma Vetorização TF-IDF
        self.vectorizer = TfidfVectorizer()
        self.text_matrix = self.vectorizer.fit_transform(self.df['combined_text_features'])
        
        """Normalização de características numéricas. As colunas como estão 
           talvez não fosse necessário, porém para as demais características como 
           temperatura e quantidade de alcool precise"""
        self.numeric_scaler = MinMaxScaler()
        self.numeric_features_normalized = self.numeric_scaler.fit_transform(
            self.df[self.numeric_columns].fillna(self.df[self.numeric_columns].mean())
        )

    def recommend_wines(self, input_features, top_n=5):
        """
        Recomenda vinhos baseado em características de entrada
        
        Parâmetros:
        input_features (dict): Dicionário com características de entrada
        top_n (int): Número de recomendações
        
        Retorna:
        list: Lista de IDs de vinhos recomendados
        """
        # Aqui filtraremos o que é texto e numérico. Pegamos o que foi inputado, que não é None, e criamos um dict.
        text_input = {k: v for k, v in input_features.items() 
                      if k in self.text_columns and v is not None}
        
        numeric_input = {k: v for k, v in input_features.items() 
                         if k in self.numeric_columns and v is not None}
        
        similarities = []
        
        # Similaridade textual: Fazemos um join em todos os values do dict e aplicamos o cosine similarity. 
        # Note que é pego a coluna 0 do text_matriz
        if text_input:
            input_text = ' '.join(str(v) for v in text_input.values())
            input_vector = self.vectorizer.transform([input_text])
            text_similarity = cosine_similarity(input_vector, self.text_matrix)[0]
            similarities.append(text_similarity)
        
        # Similaridade numérica: 
        if numeric_input:
            # Preparar entrada numérica: Separamos em uma lista de arrays com os 
            # valores de cada coluna e uma lista de nomes de colunas. 
            # Isso é feito a partir do dict numeric_input
            input_numeric_arr = []
            input_numeric_cols = []
            
            for col, value in numeric_input.items():
                if col in self.numeric_columns:
                    input_numeric_arr.append(value)
                    input_numeric_cols.append(col)
            
            if input_numeric_arr:
                # Normalizar entrada numérica -> Para todos os efeitos aplicamos aqui a transformação MinMax
                input_numeric_normalized = self.numeric_scaler.transform(
                    pd.DataFrame([input_numeric_arr], columns=input_numeric_cols)
                )
                
                # Calcular similaridade numérica
                numeric_similarities = []
                for normalized_row in self.numeric_features_normalized:
                    # Extrair valores correspondentes às colunas de entrada e calcular distância
                    row_subset = normalized_row[[self.numeric_columns.index(col) for col in input_numeric_cols]]
                    distance = np.linalg.norm(row_subset - input_numeric_normalized[0])
                    numeric_similarities.append(1 / (1 + distance))
                
                similarities.append(numeric_similarities)
        
        # Aqui acessamos as similaridades e filtramos os ids de cada vinho na nossa db
        if similarities:
            final_similarity = np.mean(similarities, axis=0)
            top_indices = final_similarity.argsort()[-top_n:][::-1]
            return self.df.iloc[top_indices]['id'].tolist()
        
        return []

# Função de conversão de DataFrame para garanir que ele terá as features necessárias  
# Muito provavelmente estará deprecado e não precisaremos mais devido a 
# feature engineering para garantir os inptus corretos
def converter_dataframe(df_original):
    df = df_original.copy()
    colunas_numericas = ['sugar_tasting', 'acidity_tasting', 'tannin_tasting', 'technical_sheet_alcohol_content']
    for col in colunas_numericas:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.replace('None', np.nan)
    
    return df


In [309]:
df_processado = converter_dataframe(db)
recommender = WineRecommender(df_processado)
recomendacoes = recommender.recommend_wines({
    'fruit_tasting':1,
    'sugar_tasting':2, 
    'acidity_tasting':2, 
    'tannin_tasting':3,
    'technical_sheet_country':'Portugal'
})
db[db['id'].isin(recomendacoes)]



,id,product_type,product_name,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,specialist_review_content,specialist_review_owner,specialist_review_occupation,combined_text_features
12,3,Vinho tinto português,Foral D. Henrique Reserva Dão DOC 2021,Vermelho-rubi,Aroma intenso de compota de frutos silvestres ...,"Paladar equilibrado, taninos sedosos e final l...",3,1,3,3,"Carnes de caça, Carnes vermelhas, Queijos",Vinho tinto,750ml,Rolha,18ºC,Portugal,Dão,12.5%,"Jaén, Touriga Nacional, Aragonez",Adega Cooperativa Mangualde,2021,2025,6 meses em tanques de aço inoxidável e barrica...,None,Ari Gorenstein,Sommelier da evino,Foral D. Henrique Reserva Dão DOC 2021 Vermelh...
360,33,Vinho tinto português,Solouro Tinto,Vermelho-granada intenso,Notas intensas de frutas vermelhas e pretas ma...,"Encorpado, redondo e frutado, com taninos marc...",3,1,3,3,"Carnes de caça, Carnes vermelhas, Pizzas e mas...",Vinho tinto,750ml,Rolha de cortiça,15ºC,Portugal,None,12%,"Touriga Franca, Tinta Roriz",Campelo,None,2025,None,None,Ari Gorenstein,Sommelier da evino,Solouro Tinto Vermelho-granada intenso Notas i...
424,140,Vinho tinto português,III Reinados Vinho Tinto,Vermelho-rubi intenso,Notas de frutas vermelhas,Macio e fresco,4,1,2,2,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,18ºC,Portugal,Barcelos,12%,"Tinta Roriz, Touriga Nacional",Caves Campelo,None,2025,None,None,Vinícius Santiago,Sommelier da evino,III Reinados Vinho Tinto Vermelho-rubi intenso...
437,172,Vinho tinto português,Influente Tinto Vinho Regional Lisboa,Vermelho-rubi,Aromas de frutas vermelhas maduras,"Harmonioso, agradável e com taninos macios",3,2,3,2,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,16ºC,Portugal,Lisboa,11.5%,"Aragonez, Blend, Castelão, Touriga Nacional, U...",Adega Cooperativa do Cadaval,None,2025,3 meses em barricas de carvalho,"Descrita por Luís Camões como ""onde a terra se...",Vinícius Santiago,Sommelier da evino,Influente Tinto Vinho Regional Lisboa Vermelho...
582,569,Vinho tinto português,Portada Winemaker's Selection 2021,Vermelho-rubi,Frutas maduras como ameixa e groselha,"Encorpado, saboroso e frutado, com final levem...",4,3,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,17ºC,Portugal,Lisboa,12.5%,Uvas variadas,DFJ Vinhos,2021,2025,1 mês em garrafa,A experiência e sabedoria da DFJ deram origem ...,Vinícius Santiago,Sommelier da evino,Portada Winemaker's Selection 2021 Vermelho-ru...


# Gerando estatísticas sobre o modelo